# Convert SQLite outputs to parquet files with CytoTable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.cellprofiler_processing/cp_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
localhost240927120001
localhost240928120001
localhost240927060001
localhost240926150001


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000,
    )

print("All plates have been converted with cytotable!")


Starting conversion with cytotable for plate: localhost240927120001
Starting conversion with cytotable for plate: localhost240928120001
Starting conversion with cytotable for plate: localhost240927060001
Starting conversion with cytotable for plate: localhost240926150001
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])

    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        [
            "Nuclei_Location_Center_X",
            "Nuclei_Location_Center_Y",
            "Cells_Location_Center_X",
            "Cells_Location_Center_Y",
            "Image_Count_Cells",
        ]
        + [
            col
            for col in df.columns
            if col
            not in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
        ]
    ].rename(
        columns=lambda col: (
            "Metadata_" + col
            if col
            in [
                "Nuclei_Location_Center_X",
                "Nuclei_Location_Center_Y",
                "Cells_Location_Center_X",
                "Cells_Location_Center_Y",
                "Image_Count_Cells",
            ]
            else col
        )
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [6]:
converted_df = pd.read_parquet(
    f"./data/converted_profiles/{plate_names[1]}_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(13721, 2016)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,398.108488,397.079770,399.047591,355.078157,13,1168,localhost240928120001,f17,G10,1,...,48.271194,48.357799,6.607844,6.058847,6.126076,5.887714,4.601669,4.478892,4.545796,4.506606
1,855.599578,277.977481,809.932735,268.256144,9,2,localhost240928120001,f04,B02,1,...,56.966002,57.149212,20.243446,20.848104,20.011861,19.960999,20.807437,21.028336,21.053395,20.743538
2,704.260681,247.533802,657.274482,248.769434,5,24,localhost240928120001,f19,B03,1,...,24.981832,23.618909,4.517895,4.695692,4.753353,4.246232,19.246264,19.563467,19.389173,18.360261
3,580.929897,165.856701,596.663683,151.208875,28,50,localhost240928120001,f03,B05,1,...,41.851450,40.962070,5.569402,5.639497,5.799303,5.354111,16.738937,16.593750,16.539002,16.276492
4,421.926235,231.551148,408.041391,256.919071,16,51,localhost240928120001,f08,B05,1,...,81.325721,79.541384,22.942340,21.064127,21.897403,21.347913,11.048527,10.706962,10.848263,10.811911
